**Pipple lecture #3 - Image Recognition**

Beste Pippelaars,

Dit notebook zal door de basis functionaliteiten van keras voor het bouwen en trainen van Convolutional Neural Networks (CNNs) lopen. Het is opgebouwd voor gebruik in  Google Colaboratory: een gratis omgeving waar je kan gebruik maken van de servers van Google. 

De omgeving zal voor de meeste herkenbaar zijn, het is namelijk gebouwd op Jupyter Notebook. Het verschil is dat Google Colaboratory is geoptimaliseerd voor het gebruik i.c.m. met andere Google Services (e.g. drive). 

**Inschakelen GPU**

Google Colaboratory beschikt de functie om op een GPU server te draaien, wat het trainen van CNNs enorm versneld (https://www.quora.com/Why-are-GPUs-well-suited-to-deep-learning) . Om deze functie in te schakelen doe je het volgende in het menu:

Runtime -> Change runtime type -> Hardware accelerator: GPU

Druk nu op “connect” rechtsboven in de hoek, de notebook draait nu op een GPU server van Google!

In het Runtime menu vind je ook verder info over het restarten, resetten en runnen van de notebook (inclusief shortcuts)

**Inhoudsopgave**


1.   Loading the images 
2.   Image Data Generators
3.   CNN as feature extractor
4.   Training a CNN
5.    Challenge









---


**1. Inladen data**


Bij het gebruik maken van Keras is het belangrijk dat je de data in de juiste structuur aanlevert (dit hebben wij al voor jullie gedaan voor deze tutorial). Dit is te zien in de volgende afbeelding:

![alt text](https://cdn-images-1.medium.com/max/800/1*HpvpA9pBJXKxaPCl5tKnLg.jpeg)


Voor deze tutorial is de data set als .zip file geupload op Github. Deze kun je ophalen en uitpakken door de volgende code uit te voeren:


In [0]:
!git clone https://github.com/bartvandriel/pipple_lecture.git
import zipfile

#path to zip
local_zip = 'pipple_lecture/UC_merced_train.zip' 

#extract zip file
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

import os

base_dir = '/tmp/' + 'UC_merced_train'

#data directories
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

De bestanden worden in de /tmp/ directory geplaatst. 



*  De data set is opgesplitst in een training en validatie set. 
*  Er zijn 21 verschillende classes
* In de training set zijn er 60 afbeelding per class en in de validatie set 20 afbeelding per class

(data set source: http://weegee.vision.ucmerced.edu/datasets/landuse.html)


In [0]:
!ls '/tmp/UC_merced_train'

In [0]:
!ls '/tmp/UC_merced_train/train'

In [0]:
!ls '/tmp/UC_merced_train/train/agricultural'



---


**2. Image Data Generators**

Het direct inladen van grote hoeveelheden afbeeldingen in het werkgeheugen zorgt al snel voor problemen. Om dit te voorkomen heeft keras een aantal slimme functies en classes.

In plaats van het vooraf inladen van afbeeldingen bevat de  *ImageDataGenerator* class functies waarmee we de afbeeldingen inladen vanuit de directories op het moment dat het nodig is.

In het kort: een *ImageDataGenerator* object genereert zogenaamde batches van afbeeldingen. Daarbij kan je een hoop functies toevoegen m.b.t. data pre-processing en augmentation. 

Meer informatie: https://keras.io/preprocessing/image/#imagedatagenerator-class

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import numpy as np

seed = 42

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, #Belangrijk om de juiste pre-processing functie gebruiken bij verschillende netwerken!
    #Hier je kan data augmentatie parameters toevoegen. Kijk in de link hierboven voor de opties
)

# Note that the validation and test data should not be augmented!
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
      train_dir,  # This is the source directory for training images
      target_size=(224, 224), #target size for CNN, depends on which network is used
      batch_size=20, #batch size, maximum depends on GPU memory
      class_mode='sparse',  #labels worden gegeven als integer nummer 
      shuffle=True, #shuffle all samples, important to do at training. 
      seed=seed  # voor reproduceerbaarheid
)

# Flow validation images in batches using validation_generator
validation_generator = val_datagen.flow_from_directory(
      validation_dir, 
      target_size=(224, 224),
      batch_size=20,
      class_mode='sparse',
      shuffle=False, 
      seed=seed) 



Door de functie .next() aan te roepen genereert   `ImageDataGenerator`  een batch afbeeldingen. Hieronder wat voorbeelden om wat inzicht te geven hoe deze data eruit ziet. 

In [0]:
demo_datagen = ImageDataGenerator()
demo_generator = demo_datagen.flow_from_directory(
      train_dir,  
      batch_size=10, 
      class_mode='sparse', 
      shuffle=True,
      seed=143
)

images, labels = demo_generator.next()

Een afbeelding wordt ingeladen als numpy array met groote (pixel_height, pixel_width, channels). Waar channels normaal gesproken 3 is, een channel voor iedere kleur (Rood, Groen, Blauw -> RGB) 


In [0]:
images.shape

De labels worden omgezet tot een nummer. Meer informatie kan je opvragen door de volgende code:

In [0]:
labels

In [0]:
demo_generator.class_indices

En de data kan je natuurlijk ook gewoon plotten als afbeelding!

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(25, 10))
for i, image in enumerate(images, 1):
  plt.subplot(2, 5, i)  
  plt.grid(False)
  plt.title(list(demo_generator.class_indices)[int(labels[i-1])])
  plt.axis('off')
  plt.imshow(image/255)



---


**3. CNN as feature extractor **

Keras beschikt over verschillende pre-trained CNNs die je kunt trainen, fine-tunen of gebruiken als feature extractors. 
In dit deel zullen we een keras model gebruiken als feature extractor waarna we een SVM trainen. 

Meer voorbeelden en toepassingen te vinden op: https://keras.io/applications/


Het model dat we gebruiken is VGG16 en is getraind op ImageNet.

In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

#Load full VGG-16 architecture with imagenet weights
base_model  = VGG16(weights='imagenet', include_top=True)

#Remove top layer which is resonsible for class predictions 
model= Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

#show summary
model.summary()

Genereer features

(ps. de volgende loop is niet de meest elegante oplossing. Een andere/betere oplossing zou zijn om predict_generator te gebruiken, hiervan volgt later een voorbeeld)

In [0]:
x_train = []
y_train = []

#loop over all images
for i in range (int(train_generator.samples/train_generator.batch_size)):
  #train_generator.samples gives the number of samples in the train set
  #train_generator.batch_size gives the batch size of the generator
  
  #get batch of images
  images, labels = train_generator.next()
  #generate features
  images = model.predict(images)
  
  #add images and labels to lists
  for image, label in zip(images, labels):
    
    x_train.append(image)
    y_train.append(label)

#Convert back to numpy array
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

x_val = []
y_val = []

for i in range (int(validation_generator.samples/validation_generator.batch_size)):

  images, labels = validation_generator.next()
  images = model.predict(images)
  
  for image, label in zip(images, labels):
    
    x_val.append(image)
    y_val.append(label)
    
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)

In [0]:
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

Iedere afbeelding wordt omgezet in een feature van groote 4096 (even groot als de laatste layer van het model).
Er zitten 1260 afbeeldingen in de train set en 420 in de validatie set. 


Train svm op de features

In [0]:
from sklearn.svm import LinearSVC

clf = LinearSVC(max_iter=3000)
clf.fit(x_train, y_train)


Evalueer het netwerk met de validatie data 

In [0]:
y_pred = clf.predict(x_val)

In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred, target_names=list(validation_generator.class_indices)))

print('accuracy: ', np.mean(y_val == y_pred))

**4. Training a CNN**

Nu is het tijd om een pre-trained CNN te fine-tunen op de nieuwe data set. We gebruiken hiervoor hetzelfde netwerk als in de vorige stap: VGG16. Het verschil is nu dat we de bovenste layers niet gebruiken. In de plaats hiervan zetten wij hier nieuwe layers op (met random weights). 
Het opbouwen van een dergelijk model is te zien in de volgende code. 

(ps. de nieuwe layers die we er op zetten zijn niet hetzelfde als de layers in het oorspronkelijke model, dit om het trainen te versnellen)

https://keras.io/layers/about-keras-layers/ (kijk in de sidebar voor alle layers!)

In [0]:
from keras.layers import Dense, GlobalAveragePooling2D

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #laad VGG16 model in met weights, maar zonder bovenste lagen

#Add new layers

#base_model
x = base_model.output 
#add GAP layer
x = GlobalAveragePooling2D(name='GAP')(x)
#add Dense layer
x = Dense(1024, activation='relu', name='fc1_new')(x)
#add Dense layer
x = Dense(1024, activation='relu', name='fc2_new')(x)
#add final classification layer where size is equal to the number of classes
predictions = Dense(train_generator.num_classes, activation='softmax', name='predictions_new')(x)

model = Model(inputs=base_model.input, outputs=predictions)

We kiezen ervoor om de eerste 10 lagen van het model te *freezen*.  Het freezen van layers helpt tegen overfitting en versnelt het trainen. 
Dit kan op de volgende manier:

In [0]:
for layer in model.layers[:10]:
  layer.trainable = False

for i, layer in enumerate(model.layers):
  print(i, layer.name, layer.trainable)

Voor een model kan worden getraind moeten we het een tweetal parameters meegeven, dit doen we door het model te 'compilen'.


*   *loss*: de loss functie bepaalt het verschil tussen de voorspellingen van het model, en de werkelijke waardes 
*  *optimizer*: dit is het algortihme dat daadwerkelijk de weights optimaliseerd. De bekendste optie is Stochastic Gradient Descent




Voor meer informatie en mogelijkheden:

https://keras.io/models/sequential/


https://keras.io/optimizers/


https://keras.io/losses/

In [0]:
from keras.optimizers import SGD

model.compile(loss='sparse_categorical_crossentropy', #Loss function
              optimizer= SGD(lr=0.001), #Optimizer
              metrics=['acc']) #Metrics to show during training

Nu alle parameters zijn ingesteld. Kan het model worden getraind. Dit kan door middel van middel van .fit_generator.
Belangrijke parameters:



*   *epochs*: aantal iteraties dat het model wordt getraind
*   *steps_per_epoch*: het aantal batches  afbeeldingen per epoch dat aan het model wordt gegeven om te trainen 

voor meer informatie en opties:

https://keras.io/models/sequential/



In [0]:
training_history = model.fit_generator(train_generator, #generator for train data
                                   epochs=5, #trainings epochs
                                   steps_per_epoch=train_generator.samples/train_generator.batch_size, #steps per epoch, in each step the model is trained on a batch of images
                                   verbose=1, #print progress 
                                   validation_data=validation_generator, #generator for validation data
                                   validation_steps=validation_generator.samples/validation_generator.batch_size,
                                   )

Genereer predictions voor alle afbeeldingen in de validatie set

In [0]:
validation_generator.reset()

y_val = []
y_pred = []

for i in range (int(validation_generator.samples/validation_generator.batch_size)):

  images, labels = validation_generator.next()
  predictions = model.predict(images)
  
  #model.predict geeft een numpy array van grootte [n_images, n_classes]. 
  #waar iedere rij de kansen geeft dat een afbeelding bij de specifieke classes horen
  
  
  predictions = np.argmax(predictions, axis=1) 
  
  for label, pred in zip(labels, predictions):
    y_val.append(label)
    y_pred.append(pred)
    
y_val = np.asarray(y_val)
y_pred = np.asarray(y_pred)

evaluatie

In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred, target_names=list(validation_generator.class_indices)))

print('accuracy: ', np.mean(y_val == y_pred))

Voorbeeld gebruik `predict_generator` (https://keras.io/models/sequential/#predict_generator)

In [0]:
validation_generator.reset()

predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size, verbose=0)
print(predictions.shape)

predictions = np.argmax(predictions, axis=1) 

#validation_generator.classes geeft alle labels van de validatie set.
print('accuracy: ', np.mean(validation_generator.classes == predictions))

**5. Challenge**

Nu is het aan jullie om een zo goed mogelijk classificatie model te bouwen. Wij hebben alvast een opzet gemaakt in een andere notebook. Later op de avond zullen wij jullie de URL geven naar de test set om het model te evalueren.

# [Link naar de challenge](https://colab.research.google.com/drive/1PgBZLGbkJlRwvNK01eEUCWMw2AAa5d0u)

